# Geoencoding of Social Housing dataset

This notebook provides geocoding for the dataset provided by MM where all public houses owned by Comune di Milano are reported. The pdf can be retried at this [link](https://casa.mmspa.eu/wps/portal/pon/case/home/patrimonio/alloggi/!ut/p/z1/04_Sj9CPykssy0xPLMnMz0vMAfIjo8zifQ0sjdwtTIx83f0NzQ0c_SzcDEKCfIz8_c30w1EVWHiYGRkEeod5-AUEBxkYOhvrRxGj3wAHcDQgTj8eBVH4jQ_Xj8JnhYW3OboCC3N3oM6w4GBD08BAAzc3YwwFmGFAyBUFuaGhEQaZnumOiooA8-SPZw!!/dz/d5/L2dBISEvZ0FBIS9nQSEh/)



In [9]:
import pandas as pd
from tabula.io import read_pdf
from functools import reduce
from geopy.geocoders import Nominatim
import numpy as np

Importing and converting the dataset for pdf

In [10]:
pdf_list=read_pdf('../datasets/elenco.pdf', pages='all')

In [109]:
indirizzi_df=pd.read_csv('../datasets/indirizzi.csv')
indirizzi_df

,_id,CODICE_VIA,NUMERO,LETTERA,BARRA,BARRA2,NUMEROCOMPLETO,MUNICIPIO,RESIDENZIALE,STATOCIVICO,...,ANNO_SOPPRESSIONE,DESCRITTIVO,ANNCSU,OPENSTREETMAP,PROGANNCSU,ID_NIL,NIL,LONG_WGS84,LAT_WGS84,Location
0,1,1,15,NaN,,N06,15N06,1,0.0,Iter in corso,...,0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,1.0,DUOMO,9.188402,45.464450,"(45.4644496019, 9.18840177503)"
1,2,1,15,NaN,NaN,N05,15N05,1,0.0,Iter in corso,...,0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,1.0,DUOMO,9.188401,45.464425,"(45.4644249493, 9.18840120369999)"
2,3,1,17,NaN,NaN,N02,17N02,1,0.0,Iter in corso,...,0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,1.0,DUOMO,9.188401,45.464507,"(45.4645074643, 9.18840120369999)"
3,4,1,17,NaN,NaN,N04,17N04,1,0.0,Iter in corso,...,0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,1.0,DUOMO,9.188404,45.464620,"(45.4646199845, 9.18840357973999)"
4,5,1,13,NaN,NaN,N08,13N08,1,0.0,Iter in corso,...,0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,1.0,DUOMO,9.188402,45.464135,"(45.4641348954, 9.18840239216999)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63324,63325,9001,5,NaN,NaN,NaN,5,8,0.0,Iter in corso,...,0,EXPO 2015,EXPO 2015,EXPO DUEMILAEQUINDICI,NaN,73.0,CASCINA MERLATA,9.106231,45.520534,"(45.520533963355554, 9.106230762519976)"
63325,63326,9001,7,NaN,NaN,NaN,7,8,0.0,Iter in corso,...,0,EXPO 2015,EXPO 2015,EXPO DUEMILAEQUINDICI,NaN,73.0,CASCINA MERLATA,9.106361,45.521008,"(45.52100771095243, 9.106360718677317)"
63326,63327,9001,7,NaN,NaN,P01,7P01,8,0.0,Iter in corso,...,0,EXPO 2015,EXPO 2015,EXPO DUEMILAEQUINDICI,NaN,73.0,CASCINA MERLATA,9.106510,45.521164,"(45.52116436208506, 9.10650981676278)"
63327,63328,9001,9,NaN,NaN,NaN,9,8,0.0,Iter in corso,...,0,EXPO 2015,EXPO 2015,EXPO DUEMILAEQUINDICI,NaN,73.0,CASCINA MERLATA,9.106836,45.521303,"(45.52130273064841, 9.10683571224355)"


In [12]:
len(pdf_list)
pdf_list[1].columns
df=pd.concat(pdf_list[1:13], axis=0)
df.columns=['CODICE COMPLESSO', 'DENOMINAZIONE COMPLESSO', 'CODICE EDIFICIO',
            'DENOMINAZIONE EDIFICIO', 'INDIRIZZO', 'CIVICO', 'COMUNE', 'CONDOMINIO']
df.head()

,CODICE COMPLESSO,DENOMINAZIONE COMPLESSO,CODICE EDIFICIO,DENOMINAZIONE EDIFICIO,INDIRIZZO,CIVICO,COMUNE,CONDOMINIO
0,100017,PONTI,10001708,DEL TURCHINO 20 C-D (VIA),VIA DEL TURCHINO,20,MILANO,NaN
1,100017,PONTI,10001709,DEL TURCHINO 20 E-F (VIA),VIA DEL TURCHINO,20,MILANO,NaN
2,100017,PONTI,10001710,DEL TURCHINO 20 G (VIA),VIA DEL TURCHINO,20,MILANO,NaN
3,100017,PONTI,10001711,DEL TURCHINO 20 H (VIA),VIA DEL TURCHINO,20,MILANO,NaN
4,100017,PONTI,10001712,DEL TURCHINO 22 A (VIA),VIA DEL TURCHINO,22,MILANO,NaN


In [13]:
len(df)

980

In [14]:
df['full_address']=df['INDIRIZZO']+" "+df['CIVICO'].astype(str)+" "+df['COMUNE']
df.head()

,CODICE COMPLESSO,DENOMINAZIONE COMPLESSO,CODICE EDIFICIO,DENOMINAZIONE EDIFICIO,INDIRIZZO,CIVICO,COMUNE,CONDOMINIO,full_address
0,100017,PONTI,10001708,DEL TURCHINO 20 C-D (VIA),VIA DEL TURCHINO,20,MILANO,NaN,VIA DEL TURCHINO 20 MILANO
1,100017,PONTI,10001709,DEL TURCHINO 20 E-F (VIA),VIA DEL TURCHINO,20,MILANO,NaN,VIA DEL TURCHINO 20 MILANO
2,100017,PONTI,10001710,DEL TURCHINO 20 G (VIA),VIA DEL TURCHINO,20,MILANO,NaN,VIA DEL TURCHINO 20 MILANO
3,100017,PONTI,10001711,DEL TURCHINO 20 H (VIA),VIA DEL TURCHINO,20,MILANO,NaN,VIA DEL TURCHINO 20 MILANO
4,100017,PONTI,10001712,DEL TURCHINO 22 A (VIA),VIA DEL TURCHINO,22,MILANO,NaN,VIA DEL TURCHINO 22 MILANO


## Geoeocnoding

We geoencode the dataset using nomatim (that uses OSM data)

In [28]:
geolocator = Nominatim(user_agent="myApp")

def geo_lat(lat):
    try:
        return geolocator.geocode(lat).latitude
    except:
        return np.nan

def geo_long(long):
    try:
        return geolocator.geocode(long).longitude
    except:
        return np.nan

In [29]:
%%time
df['lat']=df['full_address'].apply(lambda x: geo_lat(x))
df['lon']=df['full_address'].apply(lambda x: geo_long(x))

CPU times: total: 2.08 s
Wall time: 16min 22s


## Not found addresses

In [30]:
df.lat.isna().sum()

116

In [127]:
df_notfound=df[df.lat.isna()]
df_notfound

,CODICE COMPLESSO,DENOMINAZIONE COMPLESSO,CODICE EDIFICIO,DENOMINAZIONE EDIFICIO,INDIRIZZO,CIVICO,COMUNE,CONDOMINIO,full_address,lat,lon
84,100027,LAMBRO,10002701,UCELLI DI NEMI GUIDO 58 (VIA),VIA UCELLI DI NEMI GUIDO,58,MILANO,NaN,VIA UCELLI DI NEMI GUIDO 58 MILANO,NaN,NaN
40,100040,STAMIRA D'ANCONA,10004001,DEI VALTORTA 57 A-B-C (VIA),VIA DEI VALTORTA,57,MILANO,SI,VIA DEI VALTORTA 57 MILANO,NaN,NaN
41,100040,STAMIRA D'ANCONA,10004002,DEI VALTORTA 57 D-E-F (VIA),VIA DEI VALTORTA,57,MILANO,SI,VIA DEI VALTORTA 57 MILANO,NaN,NaN
69,100046,CHIESA ROSSA,10004623,S. ABBONDIO 19-21-23-25 VIA),VIA SANT'ABBONDIO,19-21-23-2,MILANO,SI,VIA SANT'ABBONDIO 19-21-23-2 MILANO,NaN,NaN
21,100048,ANTONINI,10004801,NEGRI ADA 2 A-B-C (VIA),VIA NEGRI ADA,2,MILANO,SI,VIA NEGRI ADA 2 MILANO,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
69,510237,DE ANDRE',51023704,DE ANDRE' 12 (VIA) - TORRE 3,VIA DE ANDRE' FABRIZIO,12,MILANO,NaN,VIA DE ANDRE' FABRIZIO 12 MILANO,NaN,NaN
70,510237,DE ANDRE',51023705,DE ANDRE' 12 (VIA) - TORRE 4,VIA DE ANDRE' FABRIZIO,12,MILANO,NaN,VIA DE ANDRE' FABRIZIO 12 MILANO,NaN,NaN
79,510386,FRATELLI BOZZI,51038601,FRATELLI BOZZI 23 (VIA),VIA F.LLI BOZZI,23,MILANO,NaN,VIA F.LLI BOZZI 23 MILANO,NaN,NaN
87,510388,CIVITAVECCHIA,51038806,"CIVITAVECCHIA, 100-104 (AUTORIMESSA)",VIA CIVITAVECCHIA,"100,102,104",MILANO,NaN,"VIA CIVITAVECCHIA 100,102,104 MILANO",NaN,NaN


In [110]:
indirizzi_df=indirizzi_df[['NUMERO','TIPO','DENOMINAZIONE','LONG_WGS84', 'LAT_WGS84']]
indirizzi_df['INDIRIZZO']=indirizzi_df['TIPO'].str.upper()+" "+indirizzi_df['DENOMINAZIONE']+" "+indirizzi_df['NUMERO'].astype(str)
indirizzi_df=indirizzi_df[['INDIRIZZO','LONG_WGS84', 'LAT_WGS84']]
indirizzi_df=indirizzi_df.drop_duplicates(subset='INDIRIZZO')

C:\Users\iodio\AppData\Local\Temp\ipykernel_13404\3291439383.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indirizzi_df['INDIRIZZO']=indirizzi_df['TIPO'].str.upper()+" "+indirizzi_df['DENOMINAZIONE']+" "+indirizzi_df['NUMERO'].astype(str)


In [129]:
df_notfound['INDIRIZZO-N']=df_notfound['INDIRIZZO']+" "+df_notfound["CIVICO"].astype(str)
df_notfound=pd.merge(indirizzi_df, df_notfound, how='right', left_on=['INDIRIZZO'],suffixes=('_ind',''), right_on=['INDIRIZZO-N'], indicator=True)

C:\Users\iodio\AppData\Local\Temp\ipykernel_13404\2296340904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_notfound['INDIRIZZO-N']=df_notfound['INDIRIZZO']+" "+df_notfound["CIVICO"].astype(str)


In [134]:
df_notfound['lon']=df_notfound['LONG_WGS84']
df_notfound['lat']=df_notfound['LAT_WGS84']
df_notfound=df_notfound.drop(['INDIRIZZO_ind','LONG_WGS84','LAT_WGS84','INDIRIZZO-N','_merge'], axis=1)

KeyError: 'LONG_WGS84'

In [151]:
df_notfound=df_notfound[df_notfound['lat'].isna()==False]
df_notfound=df_notfound[['CODICE EDIFICIO','lat','lon']]
df=df.merge(df_notfound, on='CODICE EDIFICIO', how='left', suffixes=('', '_nf'))
df.head()

,CODICE COMPLESSO,DENOMINAZIONE COMPLESSO,CODICE EDIFICIO,DENOMINAZIONE EDIFICIO,INDIRIZZO,CIVICO,COMUNE,CONDOMINIO,full_address,lat,lon,latnf,lonnf,latnf,lonnf,lat_nf,lon_nf
0,100017,PONTI,10001708,DEL TURCHINO 20 C-D (VIA),VIA DEL TURCHINO,20,MILANO,NaN,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,NaN,NaN,NaN,NaN,NaN,NaN
1,100017,PONTI,10001709,DEL TURCHINO 20 E-F (VIA),VIA DEL TURCHINO,20,MILANO,NaN,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,NaN,NaN,NaN,NaN,NaN,NaN
2,100017,PONTI,10001710,DEL TURCHINO 20 G (VIA),VIA DEL TURCHINO,20,MILANO,NaN,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,NaN,NaN,NaN,NaN,NaN,NaN
3,100017,PONTI,10001711,DEL TURCHINO 20 H (VIA),VIA DEL TURCHINO,20,MILANO,NaN,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,NaN,NaN,NaN,NaN,NaN,NaN
4,100017,PONTI,10001712,DEL TURCHINO 22 A (VIA),VIA DEL TURCHINO,22,MILANO,NaN,VIA DEL TURCHINO 22 MILANO,45.451893,9.228053,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,510394,SENIGALLIA,51039405,SENIGALLIA - CORPO F,VIA SENIGALLIA,60/G,MILANO,NaN,VIA SENIGALLIA 60/G MILANO,45.532143,9.168970,NaN,NaN,NaN,NaN,NaN,NaN
976,510394,SENIGALLIA,51039406,SENIGALLIA - CORPO P1,VIA SENIGALLIA,60,MILANO,NaN,VIA SENIGALLIA 60 MILANO,45.532143,9.168970,NaN,NaN,NaN,NaN,NaN,NaN
977,510394,SENIGALLIA,51039407,SENIGALLIA - CORPO P2,VIA SENIGALLIA,60,MILANO,NaN,VIA SENIGALLIA 60 MILANO,45.532143,9.168970,NaN,NaN,NaN,NaN,NaN,NaN
978,510394,SENIGALLIA,51039408,SENIGALLIA - CORPO P3,VIA SENIGALLIA,60,MILANO,NaN,VIA SENIGALLIA 60 MILANO,45.532143,9.168970,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
df['lat'] = df['lat'].fillna(df['lat_nf'])
df['lon'] = df['lon'].fillna(df['lon_nf'])
df=df.iloc[:, 0:11]

In [166]:
df

,CODICE COMPLESSO,DENOMINAZIONE COMPLESSO,CODICE EDIFICIO,DENOMINAZIONE EDIFICIO,INDIRIZZO,CIVICO,COMUNE,CONDOMINIO,full_address,lat,lon
0,100017,PONTI,10001708,DEL TURCHINO 20 C-D (VIA),VIA DEL TURCHINO,20,MILANO,NaN,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053
1,100017,PONTI,10001709,DEL TURCHINO 20 E-F (VIA),VIA DEL TURCHINO,20,MILANO,NaN,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053
2,100017,PONTI,10001710,DEL TURCHINO 20 G (VIA),VIA DEL TURCHINO,20,MILANO,NaN,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053
3,100017,PONTI,10001711,DEL TURCHINO 20 H (VIA),VIA DEL TURCHINO,20,MILANO,NaN,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053
4,100017,PONTI,10001712,DEL TURCHINO 22 A (VIA),VIA DEL TURCHINO,22,MILANO,NaN,VIA DEL TURCHINO 22 MILANO,45.451893,9.228053
...,...,...,...,...,...,...,...,...,...,...,...
975,510394,SENIGALLIA,51039405,SENIGALLIA - CORPO F,VIA SENIGALLIA,60/G,MILANO,NaN,VIA SENIGALLIA 60/G MILANO,45.532143,9.168970
976,510394,SENIGALLIA,51039406,SENIGALLIA - CORPO P1,VIA SENIGALLIA,60,MILANO,NaN,VIA SENIGALLIA 60 MILANO,45.532143,9.168970
977,510394,SENIGALLIA,51039407,SENIGALLIA - CORPO P2,VIA SENIGALLIA,60,MILANO,NaN,VIA SENIGALLIA 60 MILANO,45.532143,9.168970
978,510394,SENIGALLIA,51039408,SENIGALLIA - CORPO P3,VIA SENIGALLIA,60,MILANO,NaN,VIA SENIGALLIA 60 MILANO,45.532143,9.168970


In [171]:
df.to_csv('outputs/mm_dataset.csv')